In [11]:
####################################
# 기본 옵션
####################################
max_review_length = 16  # 시계열 데이터 분할 간격 
stride = 4
N_label_num = 4
real_y_num = 300 # train data에서 '이상(1)'으로 라벨링된 총 갯수

def uniq_columns(train_dataset):
    uniq_col = []
    for i in range(train_dataset.shape[1]):
        if len(np.unique(train_dataset.iloc[:,i], axis=0)) != 1: uniq_col.append(i)
    #cols = list(range(train_dataset.shape[1]))
    #cols = [x for x in cols if x not in uniq_col]    
    return uniq_col

def slicing_data(machine_list, X_view):
    # 각 횟수에 해당하는 데이터 분할
    X_train = []
    for machine in machine_list:
        X_train.append(X_view[X_view.iloc[:,0]==machine])
    return X_train

def get_leng(X):
    # 각 횟수에 해당하는 데이터 길이
    leng = []
    for i in range(len(X)):
        leng.append(len(X[i]))
    print(leng)
    return leng

def get_analogy_data(X):
    # 분석데이터 get
    X_2 = []
    for n in range(len(X)):
        X_2.append(np.asarray(X[n].iloc[:,3:]))
    return X_2

def cut_time_series(X_2, leng, max_review_length= max_review_length, stride=stride):
    # 시계열 데이터 분할 간격
    # stride, len길이 설정 가능
    X_train2 = []
    y_train2 = []
    
    for i, num in enumerate(leng):
        h = int((num-max_review_length)/stride+1)
        for j in range(h):
            if j >= h - N_label_num:
                X_train2.append(X_2[i][stride*j:stride*j+max_review_length])
                y_train2.append(1)
            else:    
                X_train2.append(X_2[i][stride*j:stride*j+max_review_length])
                y_train2.append(0)
    print("비정상 갯수 : ", np.sum(y_train2))
    return X_train2, y_train2

def just_cut_time(X_2, leng, max_review_length):
    # 시계열 데이터 분할 간격
    # 그냥 분할
    X_train2 = []
    y_train2 = []
    
    for i, num in enumerate(leng):
        h = int(np.floor(num/max_review_length))
        for j in range(h):
            if j >= h - N_label_num:
                X_train2.append(X_2[i][max_review_length*j:max_review_length*(j+1)])
                y_train2.append(np.ones(max_review_length))
            else:    
                X_train2.append(X_2[i][max_review_length*j:max_review_length*(j+1)])
                y_train2.append(np.zeros(max_review_length))
    
    print("비정상 갯수 : ", np.sum(y_train2))
    return X_train2, y_train2

def cut_time_series_added(X_2, leng, max_review_length, stride, real_y_num):
    # gan data 생성 후 사용
    # 시계열 데이터 분할 간격
    # stride, len길이 설정 가능
    # type(X_2) = list
    X_train2 = []
    y_train2 = []

    for i, num in enumerate(leng):
        h = int((num-max_review_length)/stride+1)
        for j in range(h):
            if j >= h - N_label_num : # 마지막 고장 label
                X_train2.append(X_2[i][stride*j:stride*j+max_review_length])
                y_train2.append(1)
                for c in range(int(fake_X_N.shape[0]/real_y_num)):
                    X_train2.append(fake_X_N[c*real_y_num])
                    y_train2.append(1)


            else:    
                X_train2.append(X_2[i][stride*j:stride*j+max_review_length])
                y_train2.append(0)
    real_y_num = np.sum(y_train2)
    print("비정상 갯수 : ", real_y_num)
    return X_train2, y_train2

def cut_time_series_test(X_2, leng, max_review_length, stride):
    # test시 사용
    # 시계열 데이터 분할 간격
    # stride, len길이 설정 가능
    # type(X_2) = list
    X_train2 = []
    y_train2 = []

    for i, num in enumerate(leng):
        h = int((num-max_review_length)/stride+1)
        for j in range(h):
            if j >= h - N_label_num + 1 :
                X_train2.append(X_2[i][stride*j:stride*j+max_review_length])
                y_train2.append(1)

            else:    
                X_train2.append(X_2[i][stride*j:stride*j+max_review_length])
                y_train2.append(0)
    print("비정상 갯수 : ", np.sum(y_train2))
    return X_train2, y_train2

def scaling(X_view):
    ####################################
    # scaling
    ####################################
    #scaled_X = pd.DataFrame(preprocessing.minmax_scale(X_view.iloc[:,3:]))
    scaled_X = pd.DataFrame(preprocessing.StandardScaler().fit_transform(X_view.iloc[:,2:]))
    scaled_X = pd.concat([X_view.iloc[:,:2], scaled_X], axis=1)
    return scaled_X

In [10]:
import numpy as np
import pandas as pd
from sklearn import preprocessing

In [8]:
##### GAN DATA LOAD
fake_X_N = np.load('fake_X_N.npy')
fake_X_N = np.reshape(fake_X_N, (fake_X_N.shape[0]*fake_X_N.shape[1], fake_X_N.shape[2], fake_X_N.shape[3]))
print(fake_X_N.shape)

(2800, 16, 17)
